In [ ]:
import sys
import AE_models

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import numpy as np
import keras 
from keras import layers
import scipy.io as sio
import pandas as pd 

import matplotlib.pyplot as plt

In [ ]:
def plotLossesLayers(h_layers, dicc, carpeta, dependencia):
    plot_layers = []
    for i in range(len(h_layers)):
        plot_layers.append(h_layers[i][1])

    keys = list(dicc.keys())

    filas = int(np.sqrt(np.ceil(len(keys)/2)))*3
    columnas = int(np.sqrt(np.round(len(keys)/2)))
    
    f, axis = plt.subplots(filas, columnas, figsize=(20,20))
    index_keys = 0
    for i in range(filas):
        for j in range(columnas):
            # error in validation
            axis[i,j].plot(plot_layers, dicc[keys[index_keys]][0])
            # error in train
            axis[i,j].plot(plot_layers, dicc[keys[index_keys]][1])
            axis[i,j].legend(["val_loss", "loss"])
            axis[i,j].set_title(keys[i])
            index_keys += 1

    plt.close(f)


## DTW D

In [ ]:
folders = ["S1", "S2", "S3", "S4", "S5"]
normalizar = True
plotAndSaveImg = False
dependencia = "DTW_D"

errores1 = []
errores2 = []

for index_data in range(len(folders)):

    print("================================================================>" + folders[index_data] + "<=======================================================")
    X_train_original, X_test_original, y_train_original, y_test_original = AE_models.loadData(index_data, normalizar, folders, dependencia)
    
    autoencodertype = {'DAE': True, 'AE': False}

    epochs = 5000
    initial_learning_rate = [1e-4, 1e-3, 1e-2, 1e-1]
    batch_size = [32]
    dropout=[0, .05, .1]
    std_noise = [0.01, 0.025, 0.05]
    h_layers_1 = [[X_train_original.shape[0], 450],
                [X_train_original.shape[0], 425],
                [X_train_original.shape[0], 400],
                [X_train_original.shape[0], 375],
                [X_train_original.shape[0],  350],
                [X_train_original.shape[0],  325],
                [X_train_original.shape[0],  300],
                [X_train_original.shape[0], 275],
                [X_train_original.shape[0], 250],
                [X_train_original.shape[0], 225],
                [X_train_original.shape[0], 200]]


    hyperparameters = {'epochs': epochs,
                       'initial_learning_rate':initial_learning_rate,
                       'kfold':5,
                       "mindelta": 0.0001}

    y_pre_train = y_train_original
    y_test = y_test_original
    X_pre_train = X_train_original
    X_test = X_test_original

    semillas = [9,18,35, 52, 75]

    kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=semillas[index_data])
    kf.get_n_splits(X_pre_train)

    bestHyperparameters = {'h_layers': -1, 'batch_size': -1, 'initial_learning_rate': -1 }

    bestMetricDev = np.inf
    dicc_1 = {}

    for init in range(len(initial_learning_rate)):
        for std in range(len(std_noise)):
            for bs in range(len(batch_size)):
                for d in range(len(dropout)):
                    plot_val_loss = []
                    plot_loss = []
                    for l in range(len(h_layers_1)):
                        hyperparameters_aux = {'epochs': hyperparameters['epochs'],
                                           'initial_learning_rate': hyperparameters['initial_learning_rate'][init],
                                           "mindelta": hyperparameters["mindelta"],
                                           'batch_size': batch_size[bs],
                                           'h_layers': h_layers_1[l],
                                           'dropout': dropout[d],
                                           'std_noise': std_noise[std],
                                           'verbose': 0}

                        v_early = []
                        v_metric_dev = []
                        v_hist = []
                        v_val_loss = []
                        for train_index, val_index in kf.split(X_pre_train):
                            X_train, X_val = X_pre_train[train_index], X_pre_train[val_index]
                            # Reset keras
                            AE_models.reset_keras()
                            # Train the network and test it
                            model, hist, early = AE_models.runNetwork(X_train, X_val,
                                                                      hyperparameters_aux,
                                                                      autoencodertype)

                            v_early.append(early)
                            v_hist.append(hist)
                            v_val_loss.append(np.min(hist.history["val_loss"]))
                        metric_dev = np.mean(v_val_loss)
                        plot_val_loss.append(metric_dev)
                        plot_loss.append(np.mean(hist.history["loss"]))

                        if metric_dev < bestMetricDev:
                            print("\tChange the best ", bestMetricDev, " by metric dev: ", metric_dev)
                            bestMetricDev = metric_dev
                            bestHyperparameters['h_layers'] = l
                            bestHyperparameters['batch_size'] = bs
                            bestHyperparameters['initial_learning_rate'] = init
                            bestHyperparameters['std_noise'] = std
                            bestHyperparameters['dropout'] = d
                            print("%%%%%%%%%bestHyperparameters%%%%%%%%%")
                            print(bestHyperparameters)

                    dicc_1["bs:" + str(batch_size[bs]) + "-lr:" + str(initial_learning_rate[init]) + "-std:" + str(std_noise[std])] = [plot_val_loss, plot_loss]

    if plotAndSaveImg:
        plotLossesLayers(h_layers_1, dicc_1, folders[index_data], dependencia)
        
    print("Layers selected:", h_layers[bestHyperparameters["h_layers"]])
    print("batch_size selected:", batch_size[bestHyperparameters["batch_size"]])
    print("initial_learning_rate selected:", initial_learning_rate[bestHyperparameters["initial_learning_rate"]])
    print("std_noise selected:", std_noise[bestHyperparameters["std_noise"]])
    print("dropout selected:", dropout[bestHyperparameters["dropout"]])

    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%================================================================> BEST MODEL SELECT WITH DIFFERENTS HYPERPARAMETERS <=======================================================%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    hyperparameters = {'epochs': hyperparameters['epochs'],
                   'initial_learning_rate': initial_learning_rate[bestHyperparameters["initial_learning_rate"]],
                   "mindelta": hyperparameters["mindelta"],
                   'batch_size': batch_size[bestHyperparameters["batch_size"]],
                   'h_layers': h_layers_1[bestHyperparameters["h_layers"]],
                   'std_noise': std_noise[bestHyperparameters["std_noise"]],
                   'dropout': dropout[bestHyperparameters["dropout"]],
                   'verbose': 1}
    
    AE_models.reset_keras()
    X_train_aux, X_val_aux = train_test_split(X_pre_train,
                                              test_size=0.3,
                                              random_state=semillas[index_data])
    
    autoencoder, hist, early = AE_models.runNetwork(X_train_aux, X_val_aux,
                                                    hyperparameters,
                                                    autoencodertype)
    
    denominador = 0
    X_val_aux = pd.DataFrame(X_val_aux)
    for i in range(X_val_aux.shape[1]):
        denominador += np.mean(np.square(X_val_aux.iloc[:, i].values))
    
    errores1.append((hist.history["val_loss"][-2]/((1/X_val_aux.shape[1])*denominador))*100)
    errores2.append((hist.history["val_loss"][-2]/((1/X_val_aux.shape[0])*denominador))*100)


    # Group the layers in an object (input and output)
    encoder = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)

    X_train = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[index_data] + '/X_train.csv')
    X_test = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[index_data] + '/X_test.csv')

    X_train, X_test = AE_models.normData_minmax(X_train, X_test)

    encoder = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    #encode the data
    X_train_encode = encoder.predict(X_train)
    X_test_encode = encoder.predict(X_test)

    pd.DataFrame(X_train_encode).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/DAE/X_train_Norm_DAE_" + folders[index_data] + ".csv", index=False)
    pd.DataFrame(X_test_encode).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/DAE/X_test_Norm_DAE_" + folders[index_data] + ".csv", index=False)

## DTW I

In [ ]:
folders_I = ["S1", "S2", "S3", "S4", "S5"]
normalizar_I = True
plotAndSaveImg_I = False
dependencia_I = "DTW_I"

for index_data_I in range(len(folders_I)):

    print("================================================================>" + folders_I[index_data_I] + "<=======================================================")
    X_train_original_I, X_test_original_I, y_train_original_I, y_test_original_I = AE_models.loadData(index_data_I, normalizar_I, folders_I, dependencia_I)

    autoencodertype_I = {'DAE': True, 'AE': False}

    epochs_I = 5000
    initial_learning_rate_I = [1e-4, 1e-3, 1e-2, 1e-1]
    batch_size_I = [32]
    dropout_I = [0, .05, .1]
    std_noise_I = [0.01, 0.025, 0.05]
    h_layers_I = [[X_train_original_I.shape[0], 450],
                [X_train_original_I.shape[0], 425],
                [X_train_original_I.shape[0], 400],
                [X_train_original_I.shape[0], 375],
                [X_train_original_I.shape[0],  350],
                [X_train_original_I.shape[0],  325],
                [X_train_original_I.shape[0],  300],
                [X_train_original_I.shape[0], 275],
                [X_train_original_I.shape[0], 250],
                [X_train_original_I.shape[0], 225],
                [X_train_original_I.shape[0], 200]]


    hyperparameters_I = {'epochs': epochs_I,
                         'initial_learning_rate': initial_learning_rate_I,
                         'kfold': 5,
                         "mindelta": 0.0001}

    y_pre_train_I = y_train_original_I
    y_test_I = y_test_original_I
    X_pre_train_I = X_train_original_I
    X_test_I = X_test_original_I

    semillas_I = [9, 18, 35, 52, 75]

    kf_I = KFold(n_splits=hyperparameters_I["kfold"], shuffle=True, random_state=semillas_I[index_data_I])
    kf_I.get_n_splits(X_pre_train_I)

    bestHyperparameters_I = {'h_layers': -1, 'batch_size': -1, 'initial_learning_rate': -1 }

    bestMetricDev_I = np.inf
    dicc_I = {}

    for init_I in range(len(initial_learning_rate_I)):
        for std_I in range(len(std_noise_I)):
            for bs_I in range(len(batch_size_I)):
                for d_I in range(len(dropout_I)):
                    plot_val_loss_I = []
                    plot_loss_I = []
                    for l_I in range(len(h_layers_I)):
                        hyperparameters_aux_I = {'epochs': hyperparameters_I['epochs'],
                                                 'initial_learning_rate': hyperparameters_I['initial_learning_rate'][init_I],
                                                 "mindelta": hyperparameters_I["mindelta"],
                                                 'batch_size': batch_size_I[bs_I],
                                                 'h_layers': h_layers_I[l_I],
                                                 'dropout': dropout_I[d_I],
                                                 'std_noise': std_noise_I[std_I],
                                                 'verbose': 0}

                        v_early_I = []
                        v_metric_dev_I = []
                        v_hist_I = []
                        v_val_loss_I = []
                        for train_index_I, val_index_I in kf_I.split(X_pre_train_I):
                            X_train_I, X_val_I = X_pre_train_I[train_index_I], X_pre_train_I[val_index_I]
                            # Reset keras
                            AE_models.reset_keras()
                            # Train the network and test it
                            model_I, hist_I, early_I = AE_models.runNetwork(X_train_I, X_val_I,
                                                                      hyperparameters_aux_I,
                                                                      autoencodertype_I)

                            v_early_I.append(early_I)
                            v_hist_I.append(hist_I)
                            v_val_loss_I.append(np.min(hist_I.history["val_loss"]))
                        metric_dev_I = np.mean(v_val_loss_I)
                        plot_val_loss_I.append(metric_dev_I)
                        plot_loss_I.append(np.mean(hist_I.history["loss"]))

                        if metric_dev_I < bestMetricDev_I:
                            print("\tChange the best ", bestMetricDev_I, " by metric dev: ", metric_dev_I)
                            bestMetricDev_I = metric_dev_I
                            bestHyperparameters_I['h_layers'] = l_I
                            bestHyperparameters_I['batch_size'] = bs_I
                            bestHyperparameters_I['initial_learning_rate'] = init_I
                            bestHyperparameters_I['std_noise'] = std_I
                            bestHyperparameters_I['dropout'] = d_I
                            print("%%%%%%%%%bestHyperparameters%%%%%%%%%")
                            print(bestHyperparameters_I)

                    dicc_I["bs:" + str(batch_size_I[bs_I]) + "-lr:" + str(initial_learning_rate_I[init_I]) + "-std:" + str(std_noise_I[std_I])] = [plot_val_loss_I, plot_loss_I]

    if plotAndSaveImg_I:
        plotLossesLayers(h_layers_I, dicc_I, folders_I[index_data_I], dependencia_I)
        
    print("Layers selected:", h_layers_I[bestHyperparameters_I["h_layers"]])
    print("batch_size selected:", batch_size_I[bestHyperparameters_I["batch_size"]])
    print("initial_learning_rate selected:", initial_learning_rate_I[bestHyperparameters_I["initial_learning_rate"]])
    print("std_noise selected:", std_noise_I[bestHyperparameters_I["std_noise"]])
    print("dropout selected:", dropout_I[bestHyperparameters_I["dropout"]])

    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%================================================================> BEST MODEL SELECT WITH DIFFERENTS HYPERPARAMETERS <=======================================================%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    hyperparameters_I = {'epochs': hyperparameters_I['epochs'],
                         'initial_learning_rate': initial_learning_rate_I[bestHyperparameters_I["initial_learning_rate"]],
                         "mindelta": hyperparameters_I["mindelta"],
                         'batch_size': batch_size_I[bestHyperparameters_I["batch_size"]],
                         'h_layers': h_layers_I[bestHyperparameters_I["h_layers"]],
                         'std_noise': std_noise_I[bestHyperparameters_I["std_noise"]],
                         'dropout': dropout_I[bestHyperparameters_I["dropout"]],
                         'verbose': 1}
    
    AE_models.reset_keras()
    X_train_aux_I, X_val_aux_I = train_test_split(X_pre_train_I,
                                                  test_size=0.3,
                                                  random_state=semillas_I[index_data_I])
    
    autoencoder_I, hist_I, early_I = AE_models.runNetwork(X_train_aux_I, X_val_aux_I,
                                                          hyperparameters_I,
                                                          autoencodertype_I)


    # Group the layers in an object (input and output)
    encoder_I = keras.Model(autoencoder_I.input, autoencoder_I.get_layer('Latent_layer').output)

    X_train_I = pd.read_csv('../data_generated_by_dtw/' + dependencia_I + '/' + folders_I[index_data_I] + '/X_train.csv')
    X_test_I = pd.read_csv('../data_generated_by_dtw/' + dependencia_I + '/' + folders_I[index_data_I] + '/X_test.csv')

    X_train_I, X_test_I = AE_models.normData_minmax(X_train_I, X_test_I)

    encoder_I = keras.Model(autoencoder_I.input, autoencoder_I.get_layer('Latent_layer').output)
    # encode the data
    X_train_encode_I = encoder_I.predict(X_train_I)
    X_test_encode_I = encoder_I.predict(X_test_I)

    pd.DataFrame(X_train_encode_I).to_csv("../1_Clasifications_models/data_reduced/" + dependencia_I + "/DAE/X_train_Norm_DAE_" + folders_I[index_data_I] + ".csv", index=False)
    pd.DataFrame(X_test_encode_I).to_csv("../1_Clasifications_models/data_reduced/" + dependencia_I + "/DAE/X_test_Norm_DAE_" + folders_I[index_data_I] + ".csv", index=False)